# 第三回レポート課題
## 1. 最もよく使うLoss関数の一つであるSoftmax with lossレイヤーを実装してください．
特にBackwardを自分で実装してみてください。式の導出は難しいので省略してokです。

In [17]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import *
import numpy as np

class MulLayer:     # 乗算レイヤ
    def __init__(
        self) -> None:
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y # xとyをひっくり返す
        dy = dout * self.x
        
        return dx, dy
    

class AddLayer:     # 加算レイヤ
    def __init__(self) -> None:
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    

class Relu:     # Reluレイヤの計算グラフ
    def __init__(self) -> None:
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)    # True/False
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    
class Sigmoid:
    def __init__(self) -> None:
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx
    
class Affine():
    
    def __init__(self, w, b) -> None:
        self.w = w
        self.b = b
        self.x = None
        self.y = None
        self.dw = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.w) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx 
        
class SoftmaxWithLoss():
    def __init__(self):
        self.loss = None
        self.y = None   # softmaxの出力
        self.t = None   # 教師データ
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.x)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

1. Two layer netの学習をしてみてください．ランダムな場合の精度が10％なのでそれ以上の精度が出るはずです.

In [18]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import *
import numpy as np

from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size, 
                 weight_init_std = 0.01) -> None:
        '''
        Args:
            input_size      : 入力層のニューロン数
            hidden_size     : 隠れ層のニューロン数
            output_size     : 出力層のニューロン数
        Returns:
            None
        '''
        # 重みの初期化
        self.params = {}
        
        # 一層目のweight, bias
        self.params["w1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        
        # 2層目のweight, bias
        self.params["w2"] = weight_init_std * np.random.randn(hidden_size, output_size, )
        self.params["b2"] = np.zeros(output_size)
        
        # レイヤの生成
        self.layers = OrderedDict() #順序付き辞書
        self.layers['Affine1'] = Affine(self.params['w1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['w2'], self.params['b2'])
        self.lastlayer = SoftmaxWithLoss()
        

    def predict(self, x):
        '''
        認識、推論を行う
        
        Args:
            x : 画像データ
        '''
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    
    def loss(self, x, t):
        '''
        損失関数の値を求める
        
        Args:
            x : 画像データ
            t : 正解ラベル
        Returns:
            y   : ndarray
        '''
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        '''
        認識精度を求める
        
        Args:
            x : 画像データ
            t : 正解ラベル
        Returns:
            y   : ndarray
        '''
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t) -> dict:
        '''
        重みパラメータに対しての勾配を求める
        これは数値微分によってパラメータの勾配を求めている、おそい
        
        Args:
            x : 画像データ
            t : 正解ラベル
        Returns:
            grads : dict
        '''
        loss_w = lambda W: self.loss(x, t)
        
        grads = {}
        # 一層目のweight, biasの勾配
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        # 二層目のweight, biasの勾配
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        '''
        重みパラメータに対する勾配を誤差逆伝播法によって求める
        '''
        # forward
        self.loss(x)
        
        # bacward
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 設定
        grads = {}
        grads['w1'] = self.layers['Affine1'].dw
        grads['b1'] = self.layers['Affine1'].db
        grads['w2'] = self.layers['Affine2'].dw
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
        

In [19]:
# データの読み込み
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
   batch_mask = np.random.choice(train_size, batch_size)
   x_batch = x_train[batch_mask]
   t_batch = t_train[batch_mask]
   # 勾配
   grad = network.numerical_gradient(x_batch, t_batch)
   #grad = gradient(x_batch, t_batch)
   #grad = gradient(network, x_batch, t_batch)
   # 更新
   for key in ('w1', 'b1', 'w2', 'b2'):
       network.params[key] -= learning_rate * grad[key]
   loss = network.loss(x_batch, t_batch)
   train_loss_list.append(loss)
   if i % iter_per_epoch == 0:
       train_acc = network.accuracy(x_train, t_train)
       test_acc = network.accuracy(x_test, t_test)
       train_acc_list.append(train_acc)
       test_acc_list.append(test_acc)
       print(train_acc, test_acc)

c:\Users\村上勘太郎\Desktop\kobo2023\repo3\..\common\functions.py:50: RuntimeWarning: invalid value encountered in log
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


0.09871666666666666 0.098


KeyboardInterrupt: 

: 